In [10]:
import pandas as pd
import json
from datetime import datetime

In [17]:
# 1. Preserve original data
raw_df = pd.read_csv('orders_raw.csv')

In [18]:
# 2. Create a working copy
df = raw_df.copy()

In [19]:
# 3. Document initial state
cleaning_log = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'original_shape': df.shape,
    'original_columns': list(df.columns),
    'steps': []
}

def log_step(description, changes):
    cleaning_log['steps'].append({
        'step': len(cleaning_log['steps']) + 1,
        'description': description,
        'changes': changes,
        'shape_after': df.shape
    })


In [20]:
# 4. Fix structural issues
# Convert order_date to datetime with mixed format handling
df['order_date'] = pd.to_datetime(df['order_date'], format='mixed')

log_step(
    "Converted order_date to datetime",
    {"mixed_formats_handled": True}
)


In [23]:
 #5. Handle missing values
missing_report = df.isnull().sum().to_dict()

# Customer ID - create placeholder for anonymous customers
df['customer_id'] = df['customer_id'].fillna('ANON-' +  (df['order_id'].astype(str).str[1:].str.zfill(3)))

# Discount - convert empty to 0 and remove %
df['discount'] = df['discount'].str.rstrip('%').fillna('0').astype(float) / 100

# Shipping city - fill with 'Unknown'
df['shipping_city'] = df['shipping_city'].fillna('Unknown')

log_step(
    "Handled missing values",
    {
        'missing_customer_ids_replaced': missing_report['customer_id'],
        'missing_discounts_set_to_zero': missing_report['discount'],
        'missing_cities_marked_unknown': missing_report['shipping_city']
    }
)

In [24]:
 #6. Standardize values
# Convert product names to title case
df['product'] = df['product'].str.title()

# Standardize city names
city_mapping = {'NYC': 'New York', 'SF': 'San Francisco'}
df['shipping_city'] = df['shipping_city'].replace(city_mapping)

# Standardize status to lowercase
df['order_status'] = df['order_status'].str.lower()

log_step(
    "Standardized values",
    {
        'products_title_cased': True,
        'city_names_standardized': list(city_mapping.keys()),
        'status_lowercased': True
    }
)

In [25]:
# 7. Validate and correct data types
# Ensure quantity is integer
df['quantity'] = pd.to_numeric(df['quantity'], downcast='integer')

# Ensure price is float
df['price'] = pd.to_numeric(df['price'], downcast='float')

log_step(
    "Validated data types",
    {
        'quantity_type': str(df['quantity'].dtype),
        'price_type': str(df['price'].dtype)
    }
)


In [26]:
# 8. Detect and handle outliers
# Calculate order totals
df['order_total'] = df['quantity'] * df['price'] * (1 - df['discount'])

# Flag potential outliers (orders over $5000)
df['is_outlier'] = df['order_total'] > 5000

log_step(
    "Identified outliers",
    {
        'outliers_found': df['is_outlier'].sum(),
        'outlier_threshold': 5000
    }
)

In [27]:
# 9. Create derived features
# Extract order day of week
df['order_dow'] = df['order_date'].dt.day_name()

# Create customer segment based on order total
df['customer_segment'] = pd.cut(
    df.groupby('customer_id')['order_total'].transform('sum'),
    bins=[-1, 500, 2000, float('inf')],
    labels=['low', 'medium', 'high']
)

log_step(
    "Created derived features",
    {
        'features_added': ['order_dow', 'customer_segment']
    }
)

In [28]:

# 10. Final validation
# Check for duplicates
duplicate_orders = df.duplicated(subset=['order_id']).sum()

# Check for negative values
negative_prices = (df['price'] < 0).sum()
negative_quantities = (df['quantity'] < 0).sum()

validation_report = {
    'duplicate_orders': duplicate_orders,
    'negative_prices': negative_prices,
    'negative_quantities': negative_quantities
}

log_step(
    "Performed final validation",
    validation_report
)


In [35]:
# 11. Save cleaned data and log
df.to_csv('orders_cleaned.csv', index=False)

# with open('cleaning_log.json') as f:
#     json.dumps(cleaning_log, indent=2)

print("Data cleaning complete!")
print(f"Original shape: {cleaning_log['original_shape']}")
print(f"Final shape: {df.shape}")

Data cleaning complete!
Original shape: (10, 9)
Final shape: (10, 13)
